# Comparing The Effectiveness Of RAG Between Models


### Dependencies

In [ ]:
!pip install --pre torch torchvision --index-url https://download.pytorch.org/whl/nightly/cu128 --quiet #!!!! nicht entfernen nur auskommentieren
!pip install sentence-transformers faiss-cpu transformers datasets transformers jupyterlab_widgets pandas numpy accelerate --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import os
import sys
#------------------------------------------------------------------------------------------------
#os.environ["HF_HOME"] = "D:/AI_Models" #-> only for my computer delete on others!!!
#------------------------------------------------------------------------------------------------
import torch
import numpy as np
from numpy.linalg import norm
import faiss
import datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from accelerate import Accelerator

c:\Users\Karl\Desktop\project\Large-Language-Modell-Project_\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(f"CUDA verfügbar: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    
    accelerator = Accelerator()
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Anzahl GPUs: {torch.cuda.device_count()}")
    print(f"CUDA Version (PyTorch): {torch.version.cuda}")
    print(f"Accelerator device: {accelerator.device}")
else:
    accelerator = Accelerator(cpu=True) 
    print(f"Accelerator device: {accelerator.device}")

CUDA verfügbar: True
GPU Name: NVIDIA GeForce RTX 2060
Anzahl GPUs: 1
CUDA Version (PyTorch): 12.8
Accelerator device: cuda


### Import the models

In [ ]:
"""
model_name_mistral = "mistralai/Mistral-7B-v0.1"
model_mistral = AutoModelForCausalLM.from_pretrained(
    model_name_mistral,
    dtype="auto"
    )
model_mistral = accelerator.prepare(model_mistral)
tokenizer_mistral = AutoTokenizer.from_pretrained(model_name_mistral)

model_name_deepseek = "deepseek-ai/deepseek-llm-7b-base"
model_deepseek = AutoModelForCausalLM.from_pretrained(
    model_name_deepseek,
    dtype="auto"
    )
model_deepseek = accelerator.prepare(model_deepseek)
tokenizer_deepseek = AutoTokenizer.from_pretrained(model_name_deepseek)

model_name_llama = "meta-llama/Llama-2-7b"
model_llama = AutoModelForCausalLM.from_pretrained(
    model_name_llama,
    dtype="auto"
    )
model_llama = accelerator.prepare(model_llama)
tokenizer_llama = AutoTokenizer.from_pretrained(model_name_llama)
"""

#"Qwen/Qwen2.5-0.5B" -> "Qwen/Qwen2-7B"
model_name_qwen = "Qwen/Qwen3-1.7B"
model_qwen = AutoModelForCausalLM.from_pretrained(
    model_name_qwen,
    dtype="auto"
)
model_qwen = accelerator.prepare(model_qwen) 
tokenizer_qwen = AutoTokenizer.from_pretrained(model_name_qwen)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 43.94it/s]


### Import the dataset

In [ ]:
"""
trivia_qa = datasets.load_dataset("mandarjoshi/trivia_qa", "rc", split="train")

documents_trivia_qa = [item["search_results"]["context"][0] for item in trivia_qa if item["search_results"]["context"]]
len(documents_trivia_qa)
"""

rag_dataset_1200 = datasets.load_dataset("neural-bridge/rag-dataset-1200", split="train")

documents_rag_1200 = [item["context"] for item in rag_dataset_1200]
len(documents_rag_1200)

960

In [4]:
documents = documents_rag_1200

def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = []
for doc in documents:
    chunks.extend(chunk_text(doc))

len(chunks)

8732

In [7]:
embed_model = SentenceTransformer("all-MiniLM-L6-v2")



embeddings = embed_model.encode(
    chunks,
    show_progress_bar=True
)

embeddings = np.array(embeddings)
embeddings.shape


Batches: 100%|██████████| 273/273 [00:09<00:00, 28.27it/s]


(8732, 384)

We use the FlatL2 index to measure the similarity of the embeddings. FlatL2 ueses the euclidian distance for that.

In [8]:
print(type(embeddings))
print(len(embeddings))  
if len(embeddings) > 0:
    print(np.array(embeddings).shape)

dimension = embeddings.shape[1]


index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

index.ntotal

<class 'numpy.ndarray'>
8732
(8732, 384)


8732

In [9]:
"""def retrieve_context(question, k=5):
    q_emb = embed_model.encode([question])
    distances, indices = index.search(q_emb, len(chunks))
    
    # Filter: nur Chunks mit mehr als 50 Zeichen UND Frage-Keywords enthalten
    keywords = [w.lower() for w in question.split()]
    selected_chunks = []
    for i in indices[0]:
        chunk = chunks[i].strip()
        if len(chunk) > 50 and any(kw in chunk.lower() for kw in keywords):
            selected_chunks.append(chunk)
        if len(selected_chunks) >= k:
            break
    return "\n".join(selected_chunks)"""


'def retrieve_context(question, k=5):\n    q_emb = embed_model.encode([question])\n    distances, indices = index.search(q_emb, len(chunks))\n\n    # Filter: nur Chunks mit mehr als 50 Zeichen UND Frage-Keywords enthalten\n    keywords = [w.lower() for w in question.split()]\n    selected_chunks = []\n    for i in indices[0]:\n        chunk = chunks[i].strip()\n        if len(chunk) > 50 and any(kw in chunk.lower() for kw in keywords):\n            selected_chunks.append(chunk)\n        if len(selected_chunks) >= k:\n            break\n    return "\n".join(selected_chunks)'

Prompt similar the the one from the Ragas paper

In [10]:
def build_rag_prompt(context, question):
    return f"""Answer the question using ONLY the information from the context.
If the answer is not explicitly stated, reply with: I don't know.

Context:
{context}

Question:
{question}

Answer:
"""


In [11]:
def answer_without_context(model, tokenizer, question, max_tokens=200, do_sample=True):
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": question}
    ]
    
    text_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True,
        #-----------------------------------------------------------
        enable_thinking=False #-> falls kein thinking model löschen!
        #-----------------------------------------------------------
    )
    inputs = tokenizer(text_input, return_tensors="pt").to(accelerator.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=do_sample,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )

    generated_ids = output_ids[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

In [12]:

def answer_with_context(model, tokenizer, question, context, max_tokens=200, do_sample=True):
    rag_instruction = (
        "Answer the question using the information from the given context.\n"
        f"question: {question}\n"
        f"context: {context}"
    )

    messages = [
        {"role": "system", "content": "You are a helpful AI assistant."},
        {"role": "user", "content": rag_instruction}
    ]
    
    text_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True,
        #-----------------------------------------------------------
        enable_thinking=False #-> falls kein thinking model löschen!
        #-----------------------------------------------------------
    )
    
    inputs = tokenizer(text_input, return_tensors="pt").to(accelerator.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=do_sample,
            temperature=0.7,
            top_p=0.95,
            eos_token_id=tokenizer.eos_token_id
        )
    generated_ids = output_ids[0][inputs["input_ids"].shape[1]:]
    return tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

In [13]:
def _answer_with_context(model, tokenizer, question, max_tokens=50, k=5):
    context = retrieve_context(question, k=k)
    prompt = build_rag_prompt(context, question)
    inputs = tokenizer(prompt, return_tensors="pt")
    prompt_len = inputs["input_ids"].shape[1]

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,        # deterministisch
            temperature=0.0,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    generated = output_ids[0][prompt_len:]
    answer = tokenizer.decode(generated, skip_special_tokens=True).strip()

    # Nur erster Satz behalten
    for sep in [".", "!", "?"]:
        if sep in answer:
            answer = answer.split(sep)[0] + sep
            break

    return answer


### The three scores mentioned in the Ragas paper
In this section we implement all three scores mentioned in the Ragas paper.
We will use these to evaluate the models rag performance

$$F = \frac{|F|}{|V|}$$

In [14]:
$$CR = \frac{\text{number of extracted sentences}}{\text{total number of senctences in }c(q)}$$

In [15]:
def extract_statements(question: str, answer: str, model, tokenizer, max_tokens=800, do_sample=True):
    #Extracts statements out of the model generated answer 
    prompt = build_statement_prompt(question, answer)

    answer = answer_without_context(model, tokenizer, prompt, max_tokens, do_sample)

    statements = [line.strip() for line in answer.split("\n") if len(line.strip()) > 3]
    return statements

In [16]:
def build_faithfullness_prompt(statements):
    prompt = f"""Consider the given context and following statements, the determine whether they are supported by the information presente in the context. 
Provide a brief explanation for each statement before arriving at the verdict (Yes/No). 
Provide a final verdict for each statement in order at the end in the given format. 
Do not deviate from the specified format.
"""
    for s in statements:
        prompt += f"Statement: {s}\n"

    prompt += f"Final verdict: "
    return prompt


In [17]:
#TODO zählt aktuell die anzahl von "Yes" soll aber richtig die anzahl der supported statements zählen
def count_supported(prompt: str):
    return prompt.count("Yes")

In [18]:
def calculate_faithfulness_score(total_statements: int, suported_statements: int):
    return total_statements / suported_statements

$$AR = \frac{1}{n} \sum\limits^n_{i=1} sim(q,q_i)$$

In [19]:
def eval_similarity(q: str, q_answer: str) -> float:
    emb = embed_model.encode([q, q_answer])
    sim = np.dot(emb[0], emb[1]) / (norm(emb[0]) * norm(emb[1]))
    return (sim + 1) / 2

$$CR = \frac{\text{number of extracted sentences}}{\text{total number of senctences in }c(q)}$$

In [ ]:
def generate_questions_from_answer(answer: str, model, tokenizer, max_tokens=150):
    prompt = f"""
Generate up to 3 questions that could be answered by the following answer.
Return one question per line.

Answer:
{answer}
"""

    output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return [q.strip() for q in output.split("\n") if q.strip().endswith("?")]


In [21]:
def calculate_answer_relevance_score(question: str, generated_questions: list):
    if len(generated_questions) == 0:
        return 0.0

    score = 0
    for q in generated_questions:
        score += eval_similarity(question, q)

    return score / len(generated_questions)


In [22]:
def calculate_answer_relevance_direct(question: str, answer: str):
    """
    Misst die semantische Ähnlichkeit zwischen Frage und Antwort direkt
    und skaliert sie auf [0,1].
    """
    if not answer or answer.lower() in ["i don't know", "unknown"]:
        return 0.0


    emb = embed_model.encode([question, answer])
    
  
    sim = np.dot(emb[0], emb[1]) / (norm(emb[0]) * norm(emb[1]))
    
 
    return (sim + 1) / 2


### RAG analysis

In [ ]:
def analyse_faithfullness(model, tokenizer, do_sample=True):
    all_statements = []
    num_supported = 0
    for i in range(5):
        question = rag_dataset_1200[i]["question"]
        context = rag_dataset_1200[i]["context"]
        answer = answer_with_context(model_qwen, tokenizer_qwen, context, question)

        statements = extract_statements(question, answer, model_qwen, tokenizer_qwen)
        all_statements.append(statements)

        faithfullness_prompt = build_faithfullness_prompt(statements)
        eval = answer_with_context(model_qwen, tokenizer_qwen, context, faithfullness_prompt)
        num_supported += count_supported(eval)

    num_all_statements = len(all_statements)
    return calculate_faithfulness_score(num_all_statements, num_supported)

print(analyse_faithfullness(model_qwen, tokenizer_qwen, do_sample=False))

5 10
0.5


In [ ]:
#TODO
def analyse_answer_relevance(model, tokenizer, do_sample=True):
    pass

In [ ]:
#TODO
def analyse_context_relevance(model, tokenizer, do_sample=True):
    pass